In [18]:
# Packages
import pandas as pd
from helpers.preprocessing import *
import ast
import numpy as np

In [35]:
ROOT_DIR  = ''
IMAGE_DIR = './data/images' # directory to save images
LABEL_DIR = './data/labels' # directory to save labels
RAW_IMAGE_DIR = './data/train_images'
N_samples = 200


In [23]:
# Read labels and add column n annotations
labels_src = 'data/train.csv'

labels = pd.read_csv(labels_src)
labels['n_annotations'] = count_n_annotations(labels)
labels = labels.apply(get_path,  args=(ROOT_DIR, IMAGE_DIR, LABEL_DIR), axis=1)
labels['annotations'] = labels['annotations'].apply(lambda x: ast.literal_eval(x))
labels['bboxes'] = labels.annotations.apply(get_bbox)

In [40]:
len(os.listdir(IMAGE_DIR))

200

In [39]:
# Random sample images for training and testing from video_0 folder
if len(os.listdir(IMAGE_DIR)) != N_samples:
    print('REMOVING OLD FILES, COPYING NEW ONES')
    for file in os.listdir(IMAGE_DIR):
        os.remove(os.path.join(IMAGE_DIR, file))
    subsample_images(os.path.join(RAW_IMAGE_DIR, 'video_0'),'data/images', labels, N_samples, 0.5)

In [9]:
df = labels.copy()
df["width"]=1280
df["height"]=720

In [10]:
cnt = 0
all_bboxes = []
for row_idx in range(df.shape[0]):
    row = df.iloc[row_idx]
    image_height = row.height
    image_width  = row.width
    bboxes_coco  = np.array(row.bboxes).astype(np.float32).copy()
    num_bbox     = len(bboxes_coco)
    names        = ['cots']*num_bbox
    labels       = [0]*num_bbox
    ## Create Annotation(YOLO)
    with open(row.label_path, 'w') as f:
        if num_bbox<1:
            annot = ''
            f.write(annot)
            cnt+=1
            continue
        #bboxes_yolo  = coco2yolo(image_height, image_width, bboxes_coco)
        bboxes_yolo  = np.clip(bboxes_yolo, 0, 1)
        all_bboxes.extend(bboxes_yolo)
        for bbox_idx in range(len(bboxes_yolo)):
            annot = [str(labels[bbox_idx])]+ list(bboxes_yolo[bbox_idx].astype(str))+(['\n'] if num_bbox!=(bbox_idx+1) else [''])
            annot = ' '.join(annot)
            annot = annot.strip(' ')
            f.write(annot)
print('Missing:',cnt)

FileNotFoundError: [Errno 2] No such file or directory: '/data/labels/video_0_0.txt'